# Example notebook using the Pykonal solver for the 2D Eikonal equation

### Import modules

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import pykonal

### Define function to plot results

In [ ]:
def plot(solver):
    nx, ny = solver.pgrid[...].shape[:-1]
    fig = plt.figure(figsize=(9, 4))
    ax1 = fig.add_subplot(1, 2, 1, aspect=1)
    ax1.set_title('Velocity')
    im = ax1.pcolormesh(solver.pgrid[..., 0], solver.pgrid[..., 1], solver.vv, cmap=plt.get_cmap('jet_r'))
    cbar = fig.colorbar(im, ax=ax1, orientation='horizontal')
    cbar.set_label('Velocity [m/s]')
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.set_title('Travel time')
    im = ax2.pcolormesh(solver.pgrid[..., 0], solver.pgrid[..., 1], solver.uu, cmap=plt.get_cmap('jet_r'))
    cbar = fig.colorbar(im, ax=ax2, orientation='horizontal')
    cbar.set_label('Travel time [s]')
    for ax in (ax1, ax2):
        xmin, ymin = solver.pgrid.min_coords 
        xmax, ymax = solver.pgrid.max_coords
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.invert_yaxis()
    return(ax1, ax2)

## Initialize the solver

### Instantiate the solver and define the velocity grid

In [ ]:
solver = pykonal.EikonalSolver2D()
solver.vgrid.min_coords      = [-75, 0]   # Set the lower bound of the computational region
solver.vgrid.node_intervals  = [1, 1]     # Set the grid-node spacing along each axis 
solver.vgrid.npts            = [151, 101] # Set the number of grid nodes along each axis

### Define the propagation grid
This is where the travel-times will actually be computed. The simplest configuration is to have the propagation coincide with the velocity grid, but you may choose to make the propagation significantly denser than the velocity grid.

In [ ]:
solver.pgrid.min_coords     = solver.vgrid.min_coords
solver.pgrid.node_intervals = solver.vgrid.node_intervals
solver.pgrid.npts           = solver.vgrid.npts

### Define a template velocity model

In [ ]:
shape = solver.vgrid[...].shape[:-1]
vv = np.ones(shape, dtype=np.float32)

### Solve the trivial case
This example solves for the travel-time field generated by a single source at $(x_0, y_0, t_0) = (0, 0, 0)$ in a uniform velocity field $v(x, y) = 1$.

In [ ]:
# First, we initialize the solver's velocity model 
solver.vv = vv

# Then add a source
solver.add_source((0, 0))
# And solve
solver.solve()

# Now plot
plot(solver)

### Multiple sources
Here we add a second source at $(x_0, y_0, t_0) = (0, 50, 25)$.

In [ ]:
solver.add_source((0, 50), t0=25)
solver.solve()

plot(solver)

### Clearing the sources
Here we clear the sources, and add a new source at $(x_0, y_0, t_0) = (-50, 40, 0)$

In [ ]:
solver.clear_sources()
solver.add_source((-50, 40))
solver.solve()

plot(solver)

### Solving a slightly more intersting two-layer case
The Fast Marching Method tracks first arrivals and so naturally tracks headwaves.

In [ ]:
vv[:, 50:] = 3.5
solver.vv = vv
solver.solve()

plot(solver)

### Solving for a linear-gradient velocity field

In [ ]:
nx, ny = solver.vgrid.npts

vy = np.linspace(1, 5, ny)
for iy in range(ny):
    vv[:, iy] = vy[iy]

solver.vv = vv
solver.solve()
ax1, ax2 = plot(solver)

### Ray tracing

In [ ]:
ax1, ax2 = plot(solver)
ray = solver.trace_ray((70, 0))
ax2.plot(ray[:, 0], ray[:, 1], 'k')

### Tracing a headwave ray

In [ ]:
nx, ny = solver.vgrid.npts

vy = np.linspace(1, 5, ny)
for iy in range(ny):
    vv[:, iy] = vy[iy]

vv[:, 60:] = 5

solver.vv = vv
solver.solve()
ax1, ax2 = plot(solver)
ray = solver.trace_ray((70, 0))
ax2.plot(ray[:, 0], ray[:, 1], 'k')